In [0]:
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
from pyspark.sql.functions import *
from pyspark.sql.functions import filter
from pyspark.sql.types import *

# LOAN DATASET #

In [0]:
#read in load dataset
# File location and type
file_location = "dbfs:/user/hive/warehouse/loan_df_to_csv"
file_type = "delta"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# schema_cust = StructType([StructField("Customer_ID", StringType()),
#                      StructField("Age", IntegerType()),
#                      StructField("Gender", IntegerType()),
#                      StructField("Occupation", StringType()),
#                      StructField("Marital Status", StringType()),
#                      StructField("Family Size", IntegerType()),
#                      StructField("Income", IntegerType()),
#                      StructField("Expenditure", IntegerType()),
#                      StructField("Use Frequency", IntegerType()),
#                      StructField("Loan Category", StringType()),
#                      StructField("Loan Amount", IntegerType()),
#                      StructField("Overdue", IntegerType()),
#                      StructField("Debt Record", IntegerType()),
#                      StructField("Returned Cheque", IntegerType()),
#                      StructField("Dishonour of Bill", IntegerType()),



# ])

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type).load(file_location)
  
df.show()

+-----------+---+------+-------------------+-------------+----------+------+-----------+------------+------------------+----------+-------+----------+--------------+---------------+
|Customer_ID|Age|Gender|         Occupation|MaritalStatus|FamilySize|Income|Expenditure|UseFrequency|      LoanCategory|LoanAmount|Overdue|DebtRecord|ReturnedCheque|DishonourOfBill|
+-----------+---+------+-------------------+-------------+----------+------+-----------+------------+------------------+----------+-------+----------+--------------+---------------+
|    IB14001| 30|  MALE|       BANK MANAGER|       SINGLE|         4| 50000|      22199|           6|           HOUSING|   1000000|      5|     42898|             6|              9|
|    IB14008| 44|  MALE|          PROFESSOR|      MARRIED|         6| 51000|      19999|           4|          SHOPPING|     50000|      3|     33999|             1|              5|
|    IB14012| 30|FEMALE|            DENTIST|       SINGLE|         3| 58450|      27675|  

In [0]:
# Print the schema of the loan dataset
print(df.describe)

<bound method DataFrame.describe of DataFrame[Customer_ID: string, Age: bigint, Gender: string, Occupation: string, MaritalStatus: string, FamilySize: bigint, Income: bigint, Expenditure: bigint, UseFrequency: bigint, LoanCategory: string, LoanAmount: bigint, Overdue: bigint, DebtRecord: bigint, ReturnedCheque: bigint, DishonourOfBill: bigint]>


In [0]:
# Print the first 5 lines of the loan dataset

df.show(5)

+-----------+---+------+------------+--------------+-----------+------+-----------+-------------+-------------+-----------+-------+-----------+---------------+-----------------+
|Customer_ID|Age|Gender|  Occupation|Marital Status|Family Size|Income|Expenditure|Use Frequency|Loan Category|Loan Amount|Overdue|Debt Record|Returned Cheque|Dishonour of Bill|
+-----------+---+------+------------+--------------+-----------+------+-----------+-------------+-------------+-----------+-------+-----------+---------------+-----------------+
|    IB14001| 30|  MALE|BANK MANAGER|        SINGLE|          4| 50000|      22199|            6|      HOUSING|    1000000|      5|      42898|              6|                9|
|    IB14008| 44|  MALE|   PROFESSOR|       MARRIED|          6| 51000|      19999|            4|     SHOPPING|      50000|      3|      33999|              1|                5|
|    IB14012| 30|FEMALE|     DENTIST|        SINGLE|          3| 58450|      27675|            5|   TRAVELLING

In [0]:
# Print the number of columns in the loan dataset
print(len(df.columns))

15


In [0]:
# Print the number of rows in the loan dataset
print(df.count())

500


In [0]:
# Print the count of distinct records in the loan dataset

print(df.distinct().count())

500


In [0]:
# find the number of loans in each category
df.groupBy('LoanCategory').count().show()

+------------------+-----+
|      LoanCategory|count|
+------------------+-----+
|           HOUSING|   67|
|        TRAVELLING|   53|
|       BOOK STORES|    7|
|       AGRICULTURE|   12|
|         GOLD LOAN|   77|
|  EDUCATIONAL LOAN|   20|
|        AUTOMOBILE|   60|
|          BUSINESS|   24|
|COMPUTER SOFTWARES|   35|
|           DINNING|   14|
|          SHOPPING|   35|
|       RESTAURANTS|   41|
|       ELECTRONICS|   14|
|          BUILDING|    7|
|        RESTAURANT|   20|
|   HOME APPLIANCES|   14|
+------------------+-----+



In [0]:
print(df.filter(df.LoanAmount > 100000).count())

450


In [0]:
# find the number of people with income greater than 60000 rupees


198

In [0]:
# fidn the number of people with 2 or more returned cheques and income less than 50000
print(df.filter((df.ReturnedCheque >= 2) & (df.Income < 50000)).count())

167


In [0]:
# find the number of people with 2 or more returned cheques and are single
print(df.filter((df.ReturnedCheque >= 2) & (df.MaritalStatus == 'SINGLE')).count())

111


In [0]:
%sql
use catalog `hive_metastore`; select * from `default`.`loan_df_to_csv` limit 100;

Customer_ID,Age,Gender,Occupation,MaritalStatus,FamilySize,Income,Expenditure,UseFrequency,LoanCategory,LoanAmount,Overdue,DebtRecord,ReturnedCheque,DishonourOfBill
IB14001,30,MALE,BANK MANAGER,SINGLE,4,50000,22199,6,HOUSING,1000000,5,42898,6,9
IB14008,44,MALE,PROFESSOR,MARRIED,6,51000,19999,4,SHOPPING,50000,3,33999,1,5
IB14012,30,FEMALE,DENTIST,SINGLE,3,58450,27675,5,TRAVELLING,75000,6,20876,3,1
IB14018,29,MALE,TEACHER,MARRIED,5,45767,12787,3,GOLD LOAN,600000,7,11000,0,4
IB14022,34,MALE,POLICE,SINGLE,4,43521,11999,3,AUTOMOBILE,200000,2,43898,1,2
IB14024,55,FEMALE,NURSE,MARRIED,6,34999,19888,4,AUTOMOBILE,47787,1,50000,0,3
IB14025,39,FEMALE,TEACHER,MARRIED,6,46619,18675,4,HOUSING,1209867,8,29999,6,8
IB14027,51,MALE,SYSTEM MANAGER,MARRIED,3,49999,19111,5,RESTAURANTS,60676,8,13000,2,5
IB14029,24,FEMALE,TEACHER,SINGLE,3,45008,17454,4,AUTOMOBILE,399435,9,51987,4,7
IB14031,37,FEMALE,SOFTWARE ENGINEER,MARRIED,5,55999,23999,5,AUTOMOBILE,60999,2,0,5,3


In [0]:
# find the  number of people with expenditure over 50000 a month 
print(df.filter(df.Expenditure > 50000).show())

+-----------+---+------+---------------+-------------+----------+------+-----------+------------+------------+----------+-------+----------+--------------+---------------+
|Customer_ID|Age|Gender|     Occupation|MaritalStatus|FamilySize|Income|Expenditure|UseFrequency|LoanCategory|LoanAmount|Overdue|DebtRecord|ReturnedCheque|DishonourOfBill|
+-----------+---+------+---------------+-------------+----------+------+-----------+------------+------------+----------+-------+----------+--------------+---------------+
|    IB14158| 54|  MALE|AIRPORT OFFICER|      MARRIED|         6| 80000|      62541|           2|  AUTOMOBILE|   2045789|      1|     16599|             2|              3|
|    IB14176| 54|  MALE|AIRPORT OFFICER|      MARRIED|         6| 80000|      62541|           2|     HOUSING|   2045789|      1|     16599|             2|              3|
|    IB14204| 54|  MALE|AIRPORT OFFICER|      MARRIED|         6| 81000|      62541|           2|     DINNING|   2045789|      1|     16599|

# CREDIT CARD DATASET #

In [0]:
# Load the credit card dataset

file_cc_path = 'dbfs:/user/hive/warehouse/credit_card'
file_type = "delta"

# The applied options are for CSV files. For other file types, these will be ignored.
df_cc = spark.read.format(file_type).load(file_cc_path)
  
df_cc.show()


+---------+----------+---------+-----------+---------+------+---+------+---------+-------------+--------------+---------------+------+
|RowNumber|CustomerId|  Surname|CreditScore|Geography|Gender|Age|Tenure|  Balance|NumOfProducts|IsActiveMember|EstimatedSalary|Exited|
+---------+----------+---------+-----------+---------+------+---+------+---------+-------------+--------------+---------------+------+
|        1|  15634602| Hargrave|        619|   France|Female| 42|     2|      0.0|            1|             1|      101348.88|     1|
|        2|  15647311|     Hill|        608|    Spain|Female| 41|     1| 83807.86|            1|             1|      112542.58|     0|
|        3|  15619304|     Onio|        502|   France|Female| 42|     8| 159660.8|            3|             0|      113931.57|     1|
|        4|  15701354|     Boni|        699|   France|Female| 39|     1|      0.0|            2|             0|       93826.63|     0|
|        5|  15737888| Mitchell|        850|    Spain|F

In [0]:
# Print the schema of the credit card dataset
df_cc.printSchema()

root
 |-- RowNumber: long (nullable = true)
 |-- CustomerId: long (nullable = true)
 |-- Surname: string (nullable = true)
 |-- CreditScore: long (nullable = true)
 |-- Geography: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Age: long (nullable = true)
 |-- Tenure: long (nullable = true)
 |-- Balance: double (nullable = true)
 |-- NumOfProducts: long (nullable = true)
 |-- IsActiveMember: long (nullable = true)
 |-- EstimatedSalary: double (nullable = true)
 |-- Exited: long (nullable = true)



In [0]:
# Print the number of columns in the credit card dataset
print(len(df_cc.columns))

13


In [0]:
# Print the number of rows in the credit card dataset
print(df_cc.count())

10000


In [0]:
# Print the number of distinct records in the credit card dataset
print(df_cc.distinct().count())

10000


In [0]:
# Print the first 5 rows in the credit card dataset
print(df_cc.show(5))

+---------+----------+--------+-----------+---------+------+---+------+---------+-------------+--------------+---------------+------+
|RowNumber|CustomerId| Surname|CreditScore|Geography|Gender|Age|Tenure|  Balance|NumOfProducts|IsActiveMember|EstimatedSalary|Exited|
+---------+----------+--------+-----------+---------+------+---+------+---------+-------------+--------------+---------------+------+
|        1|  15634602|Hargrave|        619|   France|Female| 42|     2|      0.0|            1|             1|      101348.88|     1|
|        2|  15647311|    Hill|        608|    Spain|Female| 41|     1| 83807.86|            1|             1|      112542.58|     0|
|        3|  15619304|    Onio|        502|   France|Female| 42|     8| 159660.8|            3|             0|      113931.57|     1|
|        4|  15701354|    Boni|        699|   France|Female| 39|     1|      0.0|            2|             0|       93826.63|     0|
|        5|  15737888|Mitchell|        850|    Spain|Female| 4

In [0]:
# Find the number of members who are elgible for credit card
print(df_cc.filter(df_cc.Exited == 0).count())

7963


In [0]:
# Find the number of members who are  elgible and active in the bank
print(df_cc.filter((df_cc.Exited == 0) & (df_cc.IsActiveMember == 1)).count())

4416


In [0]:
# Find the credit card users in Spain 
print(df_cc.filter(df_cc.Geography == 'Spain').show())

+---------+----------+---------+-----------+---------+------+---+------+---------+-------------+--------------+---------------+------+
|RowNumber|CustomerId|  Surname|CreditScore|Geography|Gender|Age|Tenure|  Balance|NumOfProducts|IsActiveMember|EstimatedSalary|Exited|
+---------+----------+---------+-----------+---------+------+---+------+---------+-------------+--------------+---------------+------+
|        2|  15647311|     Hill|        608|    Spain|Female| 41|     1| 83807.86|            1|             1|      112542.58|     0|
|        5|  15737888| Mitchell|        850|    Spain|Female| 43|     2|125510.82|            1|             1|        79084.1|     0|
|        6|  15574012|      Chu|        645|    Spain|  Male| 44|     8|113755.78|            2|             0|      149756.71|     1|
|       12|  15737173|  Andrews|        497|    Spain|  Male| 24|     3|      0.0|            2|             0|       76390.01|     0|
|       15|  15600882|    Scott|        635|    Spain|F

In [0]:
# Find the credit card users with Estiamted Salary greater than 100000 and have exited the card
print(df_cc.filter((df_cc.EstimatedSalary > 100000) & (df_cc.Exited == 1)).count())

1044


In [0]:
# Find the credit card users with Estiamted Salary less than 100000 and have more than 1 products
print(df_cc.filter((df_cc.EstimatedSalary < 100000) & (df_cc.NumOfProducts > 1)).count())

2432


# TRANSACTION DATASET #

In [0]:
# Load the transacton dataset

file_txn_path = 'dbfs:/user/hive/warehouse/txn_updated'
file_type = 'delta'

df_t = spark.read.format(file_type).load(file_txn_path)

df_t = df_t.withColumnRenamed("TRANSACTION DETAILS", "TransactionDetails")\
           .withColumnRenamed("VALUE DATE", "ValueDate")\
           .withColumnRenamed("WITHDRAWAL AMT", "WithdrawlAmt")\
           .withColumnRenamed("DEPOSIT AMT", "DepositAmt")\
           .withColumnRenamed("BALANCE AMT", "BalanceAmt")
df_t.show()

+------------+--------------------+----------+------------+----------+----------+
|   AccountNo|  TransactionDetails| ValueDate|WithdrawlAmt|DepositAmt|BalanceAmt|
+------------+--------------------+----------+------------+----------+----------+
|409000611074|TRF FROM  Indiafo...|2017-06-29|        NULL|   1000000|   1000000|
|409000611074|TRF FROM  Indiafo...|2017-07-05|        NULL|   1000000|   2000000|
|409000611074|FDRL/INTERNAL FUN...|2017-07-18|        NULL|    500000|   2500000|
|409000611074|TRF FRM  Indiafor...|2017-08-01|        NULL|   3000000|   5500000|
|409000611074|FDRL/INTERNAL FUN...|2017-08-16|        NULL|    500000|   6000000|
|409000611074|FDRL/INTERNAL FUN...|2017-08-16|        NULL|    500000|   6500000|
|409000611074|FDRL/INTERNAL FUN...|2017-08-16|        NULL|    500000|   7000000|
|409000611074|FDRL/INTERNAL FUN...|2017-08-16|        NULL|    500000|   7500000|
|409000611074|FDRL/INTERNAL FUN...|2017-08-16|        NULL|    500000|   8000000|
|409000611074|FD

In [0]:
# Print the schema of the transacton dataset
print(df_t.printSchema())

root
 |-- AccountNo: long (nullable = true)
 |-- TransactionDetails: string (nullable = true)
 |-- ValueDate: date (nullable = true)
 |-- WithdrawlAmt: long (nullable = true)
 |-- DepositAmt: long (nullable = true)
 |-- BalanceAmt: long (nullable = true)

None


In [0]:
#COUNT OF TRANSACTION ON EVERY ACCOUNT
df_t.groupBy('AccountNo').count().show()

+------------+-----+
|   AccountNo|count|
+------------+-----+
|409000405747|   51|
|409000438611| 4588|
|409000611074| 1093|
|409000425051|  802|
|     1196711|10536|
|409000493210| 6014|
|     1196428|48779|
|409000493201| 1044|
|409000438620|13454|
|409000362497|29840|
+------------+-----+



In [0]:
# Find the Maximum withdrawal amount for each account
df_t.groupBy('AccountNo').max('WithdrawlAmt').show()

+------------+-----------------+
|   AccountNo|max(WithdrawlAmt)|
+------------+-----------------+
|409000405747|        170000000|
|409000438611|        240000000|
|409000611074|           912000|
|409000425051|        354000000|
|     1196711|        459447546|
|409000493210|         15000000|
|     1196428|        150000000|
|409000493201|          2500000|
|409000438620|        400000000|
|409000362497|        141366239|
+------------+-----------------+



In [0]:
#MINIMUM WITHDRAWAL AMOUNT OF AN ACCOUNT
df_t.groupBy('AccountNo').min('WithdrawlAmt').show()

+------------+-----------------+
|   AccountNo|min(WithdrawlAmt)|
+------------+-----------------+
|409000405747|               21|
|409000438611|                0|
|409000611074|              120|
|409000425051|                1|
|     1196711|                0|
|409000493210|                0|
|     1196428|                0|
|409000493201|                2|
|409000438620|                0|
|409000362497|                1|
+------------+-----------------+



In [0]:
#MAXIMUM DEPOSIT AMOUNT OF AN ACCOUNT
df_t.groupBy('AccountNo').max('DepositAmt').show()

+------------+---------------+
|   AccountNo|max(DepositAmt)|
+------------+---------------+
|409000405747|      202100000|
|409000438611|      170250000|
|409000611074|        3000000|
|409000425051|       15000000|
|     1196711|      500000000|
|409000493210|       15000000|
|     1196428|      211959442|
|409000493201|        1000000|
|409000438620|      544800000|
|409000362497|      200000000|
+------------+---------------+



In [0]:
#MINIMUM DEPOSIT AMOUNT OF AN ACCOUNT
df_t.groupBy('AccountNo').min('DepositAmt').show()

+------------+---------------+
|   AccountNo|min(DepositAmt)|
+------------+---------------+
|409000405747|            500|
|409000438611|              0|
|409000611074|           1320|
|409000425051|              1|
|     1196711|              1|
|409000493210|              0|
|     1196428|              1|
|409000493201|              1|
|409000438620|              0|
|409000362497|              0|
+------------+---------------+



In [0]:
#sum of balance in every bank account
df_t.groupBy('AccountNo').sum('BalanceAmt').show()

+------------+---------------+
|   AccountNo|sum(BalanceAmt)|
+------------+---------------+
|409000405747|   -24310804720|
|409000438611| -2494865770883|
|409000611074|     1615533622|
|409000425051|    -3772118247|
|     1196711|-16047649810122|
|409000493210| -3275849521318|
|     1196428|-81418498130721|
|409000493201|     1042083205|
|409000438620| -7122918680247|
|409000362497|-52860004792808|
+------------+---------------+



In [0]:
#Number of transaction on each date
df_t.groupBy('ValueDate').count().show()

+----------+-----+
| ValueDate|count|
+----------+-----+
|2018-05-28|   80|
|2018-08-10|  216|
|2017-08-11|  128|
|2017-09-11|  109|
|2016-03-01|  109|
|2015-03-09|    8|
|2015-05-19|   17|
|2018-03-17|   50|
|2018-06-06|   84|
|2016-04-25|  161|
|2017-01-06|  111|
|2018-06-26|  117|
|2018-08-08|  174|
|2018-09-01|  194|
|2018-10-05|  132|
|2018-11-02|  121|
|2016-07-26|   98|
|2016-08-31|   74|
|2016-10-03|   95|
|2016-05-03|  105|
+----------+-----+
only showing top 20 rows



In [0]:
#List of customers with withdrawal amount more than 1 lakh
df_t.filter(df_t.WithdrawlAmt > 100000).select('AccountNo', 'TransactionDetails', 'WithdrawlAmt').show()

+------------+--------------------+------------+
|   AccountNo|  TransactionDetails|WithdrawlAmt|
+------------+--------------------+------------+
|409000611074|INDO GIBL Indiafo...|      133900|
|409000611074|INDO GIBL Indiafo...|      195800|
|409000611074|INDO GIBL Indiafo...|      143800|
|409000611074|INDO GIBL Indiafo...|      331650|
|409000611074|INDO GIBL Indiafo...|      129000|
|409000611074|INDO GIBL Indiafo...|      230013|
|409000611074|INDO GIBL Indiafo...|      367900|
|409000611074|INDO GIBL Indiafo...|      108000|
|409000611074|INDO GIBL Indiafo...|      141000|
|409000611074|INDO GIBL Indiafo...|      206000|
|409000611074|INDO GIBL Indiafo...|      242300|
|409000611074|INDO GIBL Indiafo...|      113250|
|409000611074|INDO GIBL Indiafo...|      206900|
|409000611074|INDO GIBL Indiafo...|      276000|
|409000611074|INDO GIBL Indiafo...|      171000|
|409000611074|INDO GIBL Indiafo...|      189800|
|409000611074|INDO GIBL Indiafo...|      271323|
|409000611074|INDO G